In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]= "0"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
import numpy as np
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import torch
from sklearn.model_selection import KFold
import joblib

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [2]:
Y = pd.read_csv('../data/Activity.csv',skipinitialspace=True, na_values='?')
Y.head()

,208Tl,212BiPo,212BiTl,212Pb,212Po,216Po,220Rn,224Ra,228Ac,228Ra,...,226Ra,230Th,234Pam,234PaU,234Th,234U,238U,40K,137Cs,7Be
0,23.0,42.8,22.2,68.8,40.3,61.4,61.3,64.7,69.8,60.5,...,498.0,488.0,0.855,485.0,488.0,532.0,530.0,200.00,0.868,78.3
1,16.0,27.3,15.7,41.5,25.9,44.3,42.1,43.3,42.6,40.2,...,427.0,408.0,0.629,424.0,402.0,433.0,425.0,30.50,0.349,55.5
2,29.6,53.3,27.1,84.2,56.5,87.2,82.3,79.1,82.7,86.6,...,230.0,231.0,0.353,237.0,224.0,212.0,218.0,56.50,0.587,43.2
3,11.8,20.5,11.8,30.7,20.0,33.3,32.9,33.6,29.2,30.0,...,254.0,244.0,0.417,281.0,244.0,258.0,282.0,157.00,0.212,37.8
4,106.0,172.0,95.8,275.0,168.0,257.0,297.0,289.0,291.0,275.0,...,215.0,205.0,0.345,212.0,203.0,225.0,204.0,1.45,1.800,59.3


In [3]:
list_isotopes = Y.columns.values.tolist()
list_isotopes[0:5]

['208Tl', '212BiPo', '212BiTl', '212Pb', '212Po']

In [4]:
loaded_data = np.load('../data/hpge-soil-gamma-41.npz')
x_train = loaded_data['x_train']
y_train = loaded_data['y_train']
x_test = loaded_data['x_test']
y_test = loaded_data['y_test']

print('x_train shape:\t', x_train.shape)
print('x_test shape:\t', x_test.shape)
print('y_train shape:\t', y_train.shape)
print('y_test shape:\t', y_test.shape)

x_train shape:	 (4800, 8192)
x_test shape:	 (1200, 8192)
y_train shape:	 (4800, 41)
y_test shape:	 (1200, 41)


In [5]:
# compare scaling methods for mlp inputs on regression problem
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import joblib
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
import matplotlib.pyplot as plt

from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, PoissonRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import Ridge

In [7]:
# prepare dataset with input and output scalers, can be none
def get_scaled_dataset(input_scaler, output_scaler, x_train, y_train, x_test, y_test, 
                       input_scaler_path='../scaler/input_scaler.sav'):
    x_train = x_train
    x_test = x_test
    y_test = y_test
    y_train = y_train

    if input_scaler is not None:
        # fit scaler
        input_scaler.fit(x_train)
        # transform training dataset
        x_train = input_scaler.transform(x_train)
        # transform test dataset
        x_test = input_scaler.transform(x_test)
        # save the scaler
        joblib.dump(input_scaler, input_scaler_path)
    
    if output_scaler is not None:
        # fit scaler
        output_scaler.fit(y_train)
        # transform training dataset
        y_train = output_scaler.transform(y_train)
        # transform test dataset
        y_test = output_scaler.transform(y_test)
        # save the scaler
        # joblib.dump(output_scaler, output_scaler_path)

    return x_train, y_train, x_test, y_test, output_scaler

In [ ]:
trainX, trainy, testX, testy, output_scaler = get_scaled_dataset(MinMaxScaler(), None, x_train, y_train, x_test, y_test)
input_shape = trainX.shape[1]
print(input_shape)

## Ridge Regression

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error


alphas = np.logspace(-5, 5, 100)
epsilon = 1e-6

def RidgecvMultiOutput(alpha, trainX, trainy_full, testX, testy_full):
    """
    Trains a Ridge model and returns the average MAE across all outputs.
    """
    model = Ridge(alpha=alpha)
    model.fit(trainX, trainy_full)
    pre = model.predict(testX)
    mae_avg = mean_absolute_error(testy_full, pre)
    return mae_avg

print("Optimizing alpha for multi-output regression...")
Ridge_results_avg = []
for a in alphas:
    mae_avg = RidgecvMultiOutput(a, trainX, trainy, testX, testy)
    Ridge_results_avg.append(mae_avg)

min_avg_mae = min(Ridge_results_avg)
min_index = Ridge_results_avg.index(min_avg_mae)
best_alpha = alphas[min_index]
print(f"Best alpha found based on average MAE: {best_alpha}")


print("Training final multi-output model with best alpha...")
final_model = Ridge(alpha=best_alpha)
final_model.fit(trainX, trainy)
val_pre_full = final_model.predict(testX)
results_list = []
print("Calculating metrics for each output using the single trained model...")

for idx, output_name in enumerate(list_dongvi):
    testy_output = testy[:, idx]
    val_pre_output = val_pre_full[:, idx] 

    MAE = mean_absolute_error(testy_output, val_pre_output)
    MSE = mean_squared_error(testy_output, val_pre_output)
    RMSE = np.sqrt(MSE)

    RMSLE = np.log(RMSE + epsilon) if RMSE >= 0 else np.nan 
    R2_Score = r2_score(testy_output, val_pre_output)
    MAPE = mean_absolute_percentage_error(testy_output + epsilon, val_pre_output)

    percentage_errors = np.abs((testy_output - val_pre_output) / (testy_output + epsilon))
    count_below_threshold = np.sum(percentage_errors < 0.15)
    percentage_below_threshold = count_below_threshold / len(testy_output) if len(testy_output) > 0 else 0

    result = {
        "Output": output_name,
        "Best Alpha (Overall)": best_alpha, 
        "R2 Score": R2_Score,
        "MAE": MAE,
        "MSE": MSE,
        "RMSE": RMSE,
        "RMSLE": RMSLE,
        "MAPE": MAPE,
        "Count < 0.15": count_below_threshold,
        "Percentage Below Threshold": percentage_below_threshold
    }
    results_list.append(result)

    print(f"  Output: {output_name}, R2: {R2_Score:.4f}, MAE: {MAE:.4f}, MSE: {MSE:.4f}, MAPE: {MAPE:.4f}, Percentage Below Threshold: {percentage_below_threshold:.4%}")

results_df = pd.DataFrame(results_list)
output_csv_path = "Results/ridge_multioutput_regression_results_v2.csv" 
results_df.to_csv(output_csv_path, index=False)

print(f"\nFinal multi-output results saved to {output_csv_path}")

In [ ]:
import joblib

model_filename = 'models/rigde_multioutput_model.sav'
joblib.dump(final_model, model_filename)

print(f"Model saved to {model_filename}")

## XGBoost

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
from xgboost import XGBRegressor
from sklearn.model_selection import ParameterGrid
from sklearn.multioutput import MultiOutputRegressor

epsilon = 1e-6

# --- GRID SEARCH FOR XGBOOST BASELINE ---
param_grid = {
    'estimator__n_estimators': [10, 20, 50, 100, 200],
    'estimator__max_depth': [3, 4, 5, 6, 7],
    'estimator__learning_rate': [0.01, 0.05, 0.1]
    # 'estimator__subsample': [0.8, 1.0],
    # 'estimator__colsample_bytree': [0.8, 1.0],
}
param_list = list(ParameterGrid(param_grid))

print("Optimizing hyperparameters for MultiOutputRegressor(XGBRegressor)...")
print(f"Grid size: {len(param_list)} combinations.")

param_search_results = []

# Khởi tạo mô hình cơ sở XGBoost một lần
base_xgb = XGBRegressor(objective='reg:squarederror', tree_method="gpu_hist") 
multi_output_model = MultiOutputRegressor(base_xgb, n_jobs=-1)


for i, params in enumerate(param_list):
    print(f"  Testing params combination {i+1}/{len(param_list)}: {params}")

    multi_output_model.set_params(**params)

    try:
        multi_output_model.fit(trainX, trainy)
        val_pre_full = multi_output_model.predict(testX)
        avg_mae = mean_absolute_error(testy, val_pre_full)
        param_search_results.append(avg_mae)
        print(f"    Average MAE: {avg_mae:.5f}")
    except Exception as e:
        print(f"    ERROR during training/prediction with params {params}: {e}")
        param_search_results.append(np.inf)

if not param_search_results or all(np.isinf(param_search_results)):
    print("\nERROR: Hyperparameter search failed for all combinations.")
    exit()

min_avg_mae = min(param_search_results)
min_indices = [i for i, mae in enumerate(param_search_results) if mae == min_avg_mae]
min_index = min_indices[0]

best_params_overall = param_list[min_index]
print(f"\nBest overall parameters found: {best_params_overall}")
print(f"Based on minimum average MAE: {min_avg_mae:.5f}")

print("\nTraining final MultiOutputRegressor model using the best parameters...")
final_base_xgb = XGBRegressor(objective='reg:squarederror', tree_method="gpu_hist")
final_multi_output_model = MultiOutputRegressor(final_base_xgb, n_jobs=-1)
final_multi_output_model.set_params(**best_params_overall)


final_multi_output_model.fit(trainX, trainy)
val_pre_full = final_multi_output_model.predict(testX) # Kết quả là 2D
results_list = []
print("\nCalculating metrics for each output using the single trained MultiOutputRegressor...")
for idx, output_name in enumerate(list_dongvi):
    testy_output = testy[:, idx]
    val_pre_output = val_pre_full[:, idx] 

    MAE = mean_absolute_error(testy_output, val_pre_output)
    MSE = mean_squared_error(testy_output, val_pre_output)
    RMSE = np.sqrt(MSE)
    RMSLE = np.log(RMSE + epsilon) if RMSE >= 0 else np.nan
    R2_Score = r2_score(testy_output, val_pre_output)
    MAPE = mean_absolute_percentage_error(testy_output + epsilon, val_pre_output)

    percentage_errors = np.abs((testy_output - val_pre_output) / (testy_output + epsilon))
    count_below_threshold = np.sum(percentage_errors < 0.15)
    percentage_below_threshold = count_below_threshold / len(testy_output) if len(testy_output) > 0 else 0

    best_params_str = str(best_params_overall)
    result = {
        "Output": output_name,
        "Best Params (Overall)": best_params_str, 
        "R2 Score": R2_Score,
        "MAE": MAE,
        "MSE": MSE,
        "RMSE": RMSE,
        "RMSLE": RMSLE,
        "MAPE": MAPE,
        "Count < 0.15": count_below_threshold,
        "Percentage Below Threshold": percentage_below_threshold
    }
    results_list.append(result)
    print(f"  Output: {output_name}, R2: {R2_Score:.4f}, MAE: {MAE:.4f}, MSE: {MSE:.4f}, MAPE: {MAPE:.4f}, Percentage Below Threshold: {percentage_below_threshold:.4%}")


results_df = pd.DataFrame(results_list)
output_csv_path = "Results/xgboost_MultiOutputRegressor_results.csv" # Đổi tên file cho rõ ràng
results_df.to_csv(output_csv_path, index=False)
print(f"\nFinal MultiOutputRegressor results saved to {output_csv_path}")

In [ ]:
import joblib

model_filename = 'models/xgboost_multioutput_model.sav'
joblib.dump(final_model, model_filename)

print(f"Model saved to {model_filename}")